<a href="https://colab.research.google.com/github/Saintone11/DataScience/blob/main/%EB%8C%80%EA%B5%AC%EA%B5%90%ED%86%B5%EC%82%AC%EA%B3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas==1.5.3
!pip install fastapi kaleido python-multipart uvicorn
!pip install mljar-supervised
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import warnings

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023) # Seed 고정

In [ ]:
%cd drive/MyDrive/open/open

/content/drive/MyDrive/open/open


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sec_light = pd.read_csv("external_open/대구 보안등 정보.csv", encoding ="cp949")
child_protect = pd.read_csv("external_open/대구 어린이 보호 구역 정보.csv", encoding ="cp949")
park_info = pd.read_csv("external_open/대구 주차장 정보.csv", encoding ="cp949")
train.head()

<ipython-input-56-05ffa47d9ccb>:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sec_light = pd.read_csv("external_open/대구 보안등 정보.csv", encoding ="cp949")


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,ACCIDENT_00003,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,ACCIDENT_00004,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,상해없음,승용,남,52세,경상,0,0,1,0,3


In [ ]:
location_pattern = r'(\S+) (\S+) (\S+)'

train[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train = train.drop(columns=['시군구'])
test[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test = test.drop(columns=['시군구'])

In [ ]:
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

sec_light[['도시', '구', '동', '번지']] = sec_light['소재지지번주소'].str.extract(location_pattern)
sec_light = sec_light.drop(columns=['소재지지번주소', '번지'])

sec_light = sec_light.groupby(['도시', '구', '동']).sum().reset_index()
sec_light.reset_index(inplace=True, drop=True)
sec_light

,도시,구,동,보안등위치명,설치개수,소재지도로명주소,위도,경도,설치연도,설치형태
0,대구광역시,남구,대명동,대명1동1대명1동2대명1동3대명1동4대명1동5대명1동6대명1동7대명1동8대명1동9대...,5377,대구광역시 남구 현충로 155대구광역시 남구 현충로31길 10-1대구광역시 남구 현...,188288.045338,675412.546828,10592289.0,한전주한전주건축물한전주한전주한전주한전주한전주전용주한전주한전주한전주한전주한전주한전주전...
1,대구광역시,남구,봉덕동,봉덕1동1봉덕1동2봉덕1동3봉덕1동4봉덕1동5봉덕1동6봉덕1동7봉덕1동8봉덕1동9봉...,1424,대구광역시 남구 봉덕로23길 22-5대구광역시 남구 봉덕로23길 16-3대구광역시 ...,49998.926442,179395.015619,2812887.0,한전주한전주건축물한전주한전주건축물한전주한전주한전주한전주한전주한전주한전주한전주건축물한...
2,대구광역시,남구,이천동,이천동1이천동2이천동3이천동4이천동5이천동6이천동7이천동8이천동9이천동10이천동11...,556,대구광역시 남구 명덕로50길 29-31대구광역시 남구 명덕로50길 29-45대구광역...,19754.558391,70859.061083,1110986.0,건축물건축물건축물건축물한전주한전주한전주건축물한전주한전주한전주건축물건축물한전주한전주건...
3,대구광역시,달서구,갈산동,이곡2동-093이곡2동-095이곡2동-096이곡2동-097이곡2동-098이곡2동-0...,349,대구광역시 달서구 달구벌대로250길 29대구광역시 달서구 성서로67안길 10대구광역...,12509.211192,44847.619260,698000.0,0
4,대구광역시,달서구,감삼동,감삼동-001감삼동-002감삼동-003감삼동-004감삼동-005감삼동-006감삼동-...,932,대구광역시 달서구 감삼1길 17대구광역시 달서구 감삼1길 26대구광역시 달서구 감삼...,33412.131537,119801.293525,1863897.0,0
...,...,...,...,...,...,...,...,...,...,...
223,대구광역시,중구,태평로2가,태평로2가 14-3태평로2가 19성내2동 21-3대성내2동 21-3대성내2동 21-...,38,0,1363.274654,4886.492291,64258.0,한전주건축물건축물건축물건축물건축물건축물건축물한전주한전주건축물한전주건축물건축물건축물건...
224,대구광역시,중구,태평로3가,태평로3가 214-34태평로3가 218-16태평로3가 213-8태평로3가 213-8...,47,0,1686.223047,6043.586974,50130.0,한전주건축물전용주건축물한전주한전주한전주한전주한전주한전주건축물건축물건축물건축물건축물건...
225,대구광역시,중구,포정동,포정동 75-3포정동 3-2성내1동 5-35대포정동 69-1성내1동 1-1대포정동 ...,18,0,645.694617,2314.694071,29987.0,한전주건축물건축물한전주건축물건축물건축물한전주건축물한전주건축물건축물한전주건축물건축물건축물
226,대구광역시,중구,향촌동,향촌동 10-6향촌동 14-1향촌동 44향촌동 26-1향촌동 19-3향촌동 53-6...,28,0,1004.455709,3600.623354,55953.0,건축물한전주건축물건축물한전주건축물한전주한전주한전주한전주한전주건축물건축물건축물한전주건...


In [ ]:
child_protect['보호구역수'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_protect[['도시', '구', '동', '번지']] = child_protect['소재지지번주소'].str.extract(location_pattern)
child_protect = child_protect.drop(columns=['소재지지번주소', '번지'])

child_protect = child_protect.groupby(['도시', '구', '동']).sum().reset_index()
child_protect.reset_index(inplace=True, drop=True)
child_protect

,도시,구,동,시설종류,대상시설명,소재지도로명주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자,보호구역수
0,대구광역시,남구,대명동,초등학교초등학교초등학교초등학교초등학교초등학교초등학교유치원유치원유치원유치원유치원유치원...,남도초등학교성명초등학교남덕초등학교대명초등학교남명초등학교남대구초등학교대덕초등학교계명유...,대구광역시 남구 현충동길 74(대명동)대구광역시 남구 성당로 30길 55(대명동)대...,931.947033,3342.980882,대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시대...,남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남...,YYYYYYYYYYYYYYYYYYYYYYYYYY,106.0,6~88~126~86~108~156~88~158306~886~206~156~8886...,2020-03-232020-03-232020-03-232020-03-232020-0...,26
1,대구광역시,남구,봉덕동,초등학교초등학교유치원어린이집어린이집유치원어린이집유치원,봉덕초등학교효명초등학교참좋은유치원해맑은꼬마또래어린이집꼬마또래어린이집아이린유치원참좋은...,대구광역시 남구 효성로 71(봉덕동)대구광역시 남구 앞산순환로 707대구광역시 남구...,286.713031,1028.782510,대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시,남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서,YYYYYYYY,34.0,8~1510~254010108~12406~8,2020-03-232020-03-232020-03-232020-03-232020-0...,8
2,대구광역시,남구,이천동,초등학교초등학교어린이집어린이집유치원어린이집,영선초등학교대봉초등학교호동어린이집청동어린이집드림유치원더플로우 어린이집,대구광역시 남구 영선길96(이천동)대구광역시 남구 대봉로 26길 33(이천동)대구광...,215.110868,771.611406,대구광역시대구광역시대구광역시대구광역시대구광역시대구광역시,남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서남부경찰서,YYYYYY,22.0,6~108~1710~15688,2020-03-232020-03-232020-03-232020-03-232020-0...,6
3,대구광역시,달성군,가창면,유치원유치원초등학교초등학교유치원유치원초등학교초등학교,가창초교병설용계초교병설가창초등용계초등가창초교병설용계초교병설가창초등용계초등,대구광역시 달성군 가창면 가창로 640대구광역시 달성군 가창면 가창로221길 1대구...,286.302508,1029.080164,대구광역시 달성군청대구광역시 달성군청대구광역시 달성군청대구광역시 달성군청대구광역시 ...,달성경찰서달성경찰서달성경찰서달성경찰서달성경찰서달성경찰서달성경찰서달성경찰서,NNNNNNNN,0.0,0,2019-09-102019-09-102019-09-102019-09-102019-0...,8
4,대구광역시,달성군,구지면,유치원초등학교초등학교유치원초등학교초등학교,구지초교병설구지초등세현초등학교구지초교병설구지초등세현초등학교,대구광역시 달성군 구지면 창리로11길 45-11대구광역시 달성군 구지면 창리로11길...,213.960412,770.489839,대구광역시 달성군청대구광역시 달성군청대구광역시 달성군청대구광역시 달성군청대구광역시 ...,달성경찰서달성경찰서달성경찰서달성경찰서달성경찰서달성경찰서,NYNNYN,4.0,0,2019-09-102019-09-102019-09-102019-09-102019-0...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,대구광역시,중구,봉산동,초등학교초등학교,경대사범대부설초등학교대구초등학교,대구광역시 중구 달구벌대로 2150대구광역시 중구 명륜로23길 80,71.727620,257.193436,대구광역시 중구청대구광역시 중구청,대구광역시 중부경찰서대구광역시 중부경찰서,YY,18.0,5~113~12,2019-06-012019-06-01,2
67,대구광역시,중구,삼덕동2가,초등학교,동덕초등학교,대구광역시 중구 동덕로26길 20,35.864739,128.604646,대구광역시 중구청,대구광역시 중부경찰서,Y,9.0,5~13,2019-06-01,1
68,대구광역시,중구,삼덕동3가,초등학교,삼덕초등학교,대구광역시 중구 달구벌대로447길 10,35.863588,128.607843,대구광역시 중구청,대구광역시 중부경찰서,Y,11.0,4~8,2019-06-01,1
69,대구광역시,중구,서문로1가,초등학교,종로초등학교,대구광역시 중구 경상감영길 49,35.872496,128.589707,대구광역시 중구청,대구광역시 중부경찰서,Y,8.0,6~12,2019-06-01,1


In [ ]:
park_info = pd.get_dummies(park_info, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

park_info[['도시', '구', '동', '번지']] = park_info['소재지지번주소'].str.extract(location_pattern)
park_info = park_info.drop(columns=['소재지지번주소', '번지'])

park_info = park_info.groupby(['도시', '구', '동']).sum().reset_index()
park_info.reset_index(inplace=True, drop=True)

In [ ]:
train = pd.merge(train, sec_light, how='left', on=['도시', '구', '동'])
train = pd.merge(train, child_protect, how='left', on=['도시', '구', '동'])
train = pd.merge(train, park_info, how='left', on=['도시', '구', '동'])
test = pd.merge(test, sec_light, how='left', on=['도시', '구', '동'])
test = pd.merge(test, child_protect, how='left', on=['도시', '구', '동'])
test = pd.merge(test, park_info, how='left', on=['도시', '구', '동'])

In [ ]:
train['구별평균사망자수'] = train['구'].map(train.groupby('구')['사망자수'].mean())
train['노면상태별평균사망자수'] = train['노면상태'].map(train.groupby('노면상태')['사망자수'].mean())
train['기상상태별평균사망자수'] = train['기상상태'].map(train.groupby('기상상태')['사망자수'].mean())
test["구별평균사망자수"] = train['구'].map(train.groupby('구')['사망자수'].mean())
test['노면상태별평균사망자수'] = train['노면상태'].map(train.groupby('노면상태')['사망자수'].mean())
test['기상상태별평균사망자수'] = train['기상상태'].map(train.groupby('기상상태')['사망자수'].mean())

In [ ]:
train = train[["기상상태별평균사망자수","노면상태별평균사망자수","구별평균사망자수","사고일시","요일","기상상태","구","동","도로형태","노면상태","사고유형","설치개수","보호구역수","급지구분_1","급지구분_2","급지구분_3","ECLO"]]
test = test[["기상상태별평균사망자수","노면상태별평균사망자수","구별평균사망자수","사고일시","요일","기상상태","구","동","도로형태","노면상태","사고유형","설치개수","보호구역수","급지구분_1","급지구분_2","급지구분_3"]]
road_pattern = r'(.+) - (.+)'

train[['도로형태1', '도로형태2']] = train['도로형태'].str.extract(road_pattern)
train = train.drop(columns=['도로형태'])
test[['도로형태1', '도로형태2']] = test['도로형태'].str.extract(road_pattern)
test = test.drop(columns=['도로형태'])


In [ ]:
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
train["사고일시"] = pd.to_datetime(train["사고일시"])
train["year"] = train["사고일시"].dt.year
train["month"] = train["사고일시"].dt.month
train["day"] = train["사고일시"].dt.day
train["hour"] = train["사고일시"].dt.hour

test["사고일시"] = pd.to_datetime(test["사고일시"])
test["year"] = test["사고일시"].dt.year
test["month"] = test["사고일시"].dt.month
test["day"] = test["사고일시"].dt.day
test["hour"] = test["사고일시"].dt.hour

holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']
train['holiday'] = np.where(((train.요일 == "토요일") | (train.요일 == "일요일")) | (train.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)
train['holiday'] = np.where(((train.요일 == "토요일") | (train.요일 == "일요일")) | (train.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)
train.drop(["사고일시"],axis=1,inplace=True)
test.drop(["사고일시"],axis=1,inplace=True)

In [ ]:
train['sin_hour'] = np.sin(2 * np.pi * train['hour'] / 24)
train['cos_hour'] = np.cos(2 * np.pi * train['hour'] / 24)
train.drop(["hour"],axis=1,inplace=True)
test['sin_hour'] = np.sin(2 * np.pi * test['hour'] / 24)
test['cos_hour'] = np.cos(2 * np.pi * test['hour'] / 24)
test.drop(["hour"],axis=1,inplace=True)

In [ ]:
train_y = train.pop("ECLO")

In [ ]:
encoded_train = pd.get_dummies(train["구"])
encoded_test = pd.get_dummies(test["구"])
train = train.join(encoded_train)
test = test.join(encoded_test)
train.drop(["구"],axis=1,inplace=True)
test.drop(["구"],axis=1,inplace=True)
train

,기상상태별평균사망자수,노면상태별평균사망자수,구별평균사망자수,요일,기상상태,동,노면상태,사고유형,설치개수,보호구역수,...,sin_hour,cos_hour,남구,달서구,달성군,동구,북구,서구,수성구,중구
0,0.007324,0.007359,0.011369,화요일,맑음,대신동,건조,차대사람,391.0,2.0,...,0.000000,1.000000,False,False,False,False,False,False,False,True
1,0.006859,0.007359,0.004752,화요일,흐림,감삼동,건조,차대사람,932.0,0.0,...,0.000000,1.000000,False,True,False,False,False,False,False,False
2,0.007324,0.007359,0.004324,화요일,맑음,두산동,건조,차대사람,473.0,5.0,...,0.258819,0.965926,False,False,False,False,False,False,True,False
3,0.007324,0.007359,0.008271,화요일,맑음,복현동,건조,차대차,534.0,11.0,...,0.500000,0.866025,False,False,False,False,True,False,False,False
4,0.007324,0.007359,0.010110,화요일,맑음,신암동,건조,차대차,2057.0,0.0,...,0.866025,0.500000,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,0.007324,0.007359,0.004324,금요일,맑음,수성동3가,건조,차대차,0.0,1.0,...,-0.965926,0.258819,False,False,False,False,False,False,True,False
39605,0.007324,0.007359,0.004752,금요일,맑음,상인동,건조,차대차,843.0,0.0,...,-0.965926,0.258819,False,True,False,False,False,False,False,False
39606,0.007324,0.007359,0.004752,금요일,맑음,월성동,건조,차대차,164.0,0.0,...,-0.707107,0.707107,False,True,False,False,False,False,False,False
39607,0.007324,0.007359,0.004752,금요일,맑음,장동,건조,차대차,210.0,0.0,...,-0.500000,0.866025,False,True,False,False,False,False,False,False


In [ ]:
from category_encoders.target_encoder import TargetEncoder

features = ["요일","기상상태","동","노면상태","사고유형","도로형태1","도로형태2"]

for i in features:
    encoder = TargetEncoder(cols=[i])
    train[i] = encoder.fit_transform(train[i], train_y)
    train[i] = encoder.fit_transform(train[i], train_y)
    test[i] = encoder.transform(test[i])

display(train.head())
display(test.head())

,기상상태별평균사망자수,노면상태별평균사망자수,구별평균사망자수,요일,기상상태,동,노면상태,사고유형,설치개수,보호구역수,...,sin_hour,cos_hour,남구,달서구,달성군,동구,북구,서구,수성구,중구
0,0.007324,0.007359,0.011369,4.627926,4.712888,4.282449,4.712878,3.817650,391.0,2.0,...,0.000000,1.000000,False,False,False,False,False,False,False,True
1,0.006859,0.007359,0.004752,4.627926,4.779150,4.738938,4.712878,3.817650,932.0,0.0,...,0.000000,1.000000,False,True,False,False,False,False,False,False
2,0.007324,0.007359,0.004324,4.627926,4.712888,4.842715,4.712878,3.817650,473.0,5.0,...,0.258819,0.965926,False,False,False,False,False,False,True,False
3,0.007324,0.007359,0.008271,4.627926,4.712888,4.208920,4.712878,4.944597,534.0,11.0,...,0.500000,0.866025,False,False,False,False,True,False,False,False
4,0.007324,0.007359,0.010110,4.627926,4.712888,4.549091,4.712878,4.944597,2057.0,0.0,...,0.866025,0.500000,False,False,False,True,False,False,False,False


,기상상태별평균사망자수,노면상태별평균사망자수,구별평균사망자수,요일,기상상태,동,노면상태,사고유형,설치개수,보호구역수,...,sin_hour,cos_hour,남구,달서구,달성군,동구,북구,서구,수성구,중구
0,0.007324,0.007359,0.011369,4.726704,4.726704,4.726704,4.726704,4.726704,700.0,5.0,...,0.258819,9.659258e-01,False,False,False,False,False,False,True,False
1,0.006859,0.007359,0.004752,4.726704,4.726704,4.726704,4.726704,4.726704,0.0,10.0,...,0.258819,9.659258e-01,False,False,False,False,False,False,True,False
2,0.007324,0.007359,0.004324,4.726704,4.726704,4.726704,4.726704,4.726704,0.0,1.0,...,0.866025,5.000000e-01,False,False,False,False,False,False,True,False
3,0.007324,0.007359,0.008271,4.726704,4.726704,4.726704,4.726704,4.726704,0.0,7.0,...,0.866025,5.000000e-01,False,False,False,False,False,False,True,False
4,0.007324,0.007359,0.010110,4.726704,4.726704,4.726704,4.726704,4.726704,932.0,0.0,...,1.000000,6.123234e-17,False,True,False,False,False,False,False,False


In [ ]:
def WRMSE(y_true, y_pred, weight=2):
    error = y_pred - y_true
    weighted_error = np.where(error < 0, error * weight, error)
    mse = np.mean(weighted_error**2)
    weighted_rmse = np.sqrt(mse)
    return weighted_rmse

from supervised import AutoML
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'], total_time_limit=43200, eval_metric= 'rmse', ml_task = "regression",)

In [ ]:
automl.fit(train, train_y)

AutoML directory: AutoML_9
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 3.218633 trained in 0.66 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
1_Default_LightGBM rmse 3.233743 trained in 12.58 seconds
2_Default_Xgboost rmse 3.233203 trained in 9.63 seconds
3_Default_CatBoost rmse 3.227798 trained in 22.08 seconds

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to '

3_Default_CatBoost_KMeansFeatures rmse 3.230388 trained in 39.01 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to '

25_CatBoost_KMeansFeatures rmse 3.230927 trained in 44.02 seconds


...

31_CatBoost_KMeansFeatures rmse 3.230691 trained in 92.33 seconds
* Step insert_random_feature will try to check up to 1 model
3_Default_CatBoost_RandomFeature rmse 3.227719 trained in 41.45 seconds


/usr/local/lib/python3.10/dist-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 49324 (\N{HANGUL SYLLABLE SA}) missing from current font.
/usr/local/lib/python3.10/dist-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 44256 (\N{HANGUL SYLLABLE GO}) missing from current font.
/usr/local/lib/python3.10/dist-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 50976 (\N{HANGUL SYLLABLE YU}) missing from current font.
/usr/local/lib/python3.10/dist-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 54805 (\N{HANGUL SYLLABLE HYEONG}) missing from current font.
/usr/local/lib/python3.10/dist-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 46041 (\N{HANGUL SYLLABLE DONG}) missing from current font.
/usr/local/lib/python3.10/dist-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 46020 (\N{HANGUL SYLLABLE DO}) missing from current font.
/usr/local/lib/python3.10/di

Drop features ['노면상태별평균사망자수', 'random_feature', '서구', '수성구', '급지구분_3', '남구', '설치개수', '급지구분_2', '기상상태별평균사망자수', '북구', '달성군', '달서구', '동구', '구별평균사망자수']
* Step features_selection will try to check up to 4 models
3_Default_CatBoost_SelectedFeatures rmse 3.227695 trained in 17.41 seconds
22_LightGBM_SelectedFeatures rmse 3.229894 trained in 14.85 seconds
37_RandomForest_SelectedFeatures rmse 3.230813 trained in 123.78 seconds
10_Xgboost_categorical_mix_SelectedFeatures rmse 3.230718 trained in 13.35 seconds
* Step hill_climbing_1 will try to check up to 20 models
41_CatBoost_SelectedFeatures rmse 3.227037 trained in 23.08 seconds
42_CatBoost_SelectedFeatures rmse 3.228279 trained in 14.72 seconds
43_CatBoost rmse 3.227742 trained in 30.93 seconds
44_CatBoost rmse 3.228788 trained in 19.54 seconds
45_CatBoost_GoldenFeatures rmse 3.228219 trained in 38.12 seconds
46_LightGBM_SelectedFeatures rmse 3.229945 trained in 15.21 seconds
47_LightGBM_SelectedFeatures rmse 3.229174 trained in 16.01 secon

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Custom TB Handler failed, unregistering
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/supervised/base_automl.py", line 1195, in _fit
    trained = self.train_model(params)
  File "/usr/local/lib/python3.10/dist-packages/supervised/base_automl.py", line 407, in train_model
    mf.save(results_path, model_subpath)
  File "/usr/local/lib/python3.10/dist-packages/supervised/model_framework.py", line 644, in save
    LearningCurves.plot(
  File "/usr/local/lib/python3.10/dist-packages/supervised/utils/learning_curves.py", line 46, in plot
    LearningCurves.plot_iterations(
  File "/usr/local/lib/python3.10/dist-packages/supervised/utils/learning_curves.py", line 131, in plot_iterations
    plt.savefig(plot_path)
  File "/usr/local/lib/python3.10/dist-packages/matplotlib/pyplot.py", line 1023, in savefig
    res = fig.savefig(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/matplotlib/figure.py", line 3343, in savefig
    self.canvas.print_figure(fname, **kwargs)
  File "/usr/l

In [ ]:
pred = automl.predict(test)

In [ ]:
sample = pd.read_csv("sample_submission.csv")
sample

In [ ]:
submission = sample.copy()
submission['ECLO'] = pred["prediction_label"]

In [ ]:
def custom_round(value):
    first_decimal = int((value * 10) % 10)
    if first_decimal >= 3:
        return round(value)
    else:
        return int(value)
submission['ECLO'] = submission['ECLO'].apply(custom_round)

In [ ]:
submission.to_csv("submission7.csv",encoding="utf-8",index=False)

In [ ]:
%pwd

In [ ]:
submission.shape

In [ ]:
x_train.columns